# Introduction

Welcome to NVIDIA's NVSHMEM training! In this course you will learn how to use [NVSHMEM](https://developer.nvidia.com/nvshmem), a parallel programming model for efficient and scalable communication across multiple NVIDIA GPUs. NVSHMEM, which is based on [OpenSHMEM](http://openshmem.org/site/), provides a global address space for data that spans the memory of multiple GPUs and can be accessed with fine-grained GPU-initiated operations, CPU-initiated operations, and operations on CUDA streams. NVSHMEM offers a compelling alternative to other multi-GPU programming models for many application use cases, and in this course you will compare these various multi-GPU programming models and learn about the cases where NVSHMEM makes sense to use.

We'll talk about the details later. For now, we can say that NVSHMEM is especially valuable on modern GPU servers that have a high density of GPUs per server node and complex interconnects such as [NVIDIA NVSwitch](https://www.nvidia.com/en-us/data-center/nvlink/) on the [NVIDIA DGX A100 server](https://www.nvidia.com/en-us/data-center/dgx-a100/).

<center><img src="images/NVSHMEM.png" width="1000"></center>

## Motivation

Traditionally, communication patterns involving GPUs on multiple servers may look like the following: <span style="color:limegreen">compute</span> happens on the GPU, while <span style="color:skyblue">communication</span> happens on the CPU after synchronizing the GPU (to ensure that the data we send is valid). While this approach is very easy to program, it inserts the latency of initiating the communication and/or launching the kernel on the application's critical path. We are losing out on the ability to overlap communication with compute. If we do overlap communication with compute by pipelining the work, we can partially hide the latency, but at the cost of icreased application complexity.

<center><img src="images/CPU_initiated_communication.png" width="1000"/></center>

By contrast, in a model with GPU-initiated rather than CPU-initiated communication, we do *both* compute and communication directly from the GPU. We can write extremely fine-grained communication patterns this way, and we can hide communication latency by the very nature of the GPU architecture (where warps that are computing can continue on while other warps are stalled waiting for data).

<center><img src="images/GPU_initiated_communication.png" width="1000"></center>

## Logistics

In this course you will have access to multiple NVIDIA GPUs. To see which ones are available on this node, execute the following cell (by selecting it and clicking the Run button above, or by selecting it and typing Shift + Enter). Note that any executed command starting with "!" means that we want to run the command as if we are in a terminal.

In [ ]:
!nvidia-smi

Let's store the number of devices in a variable for easy reference later.

In [7]:
NUM_DEVICES = !nvidia-smi -L | wc -l
NUM_DEVICES = int(NUM_DEVICES[0])
print("Number of devices = {}".format(NUM_DEVICES))

Number of devices = 4


## An example to warm up

Let's start with a simple parallel programming example and implement it several different ways; this will serve as a warmup exercise and give us our first introduction to NVSHMEM.

The example we'll use is the parallel calculation of the value of $\pi$. A [well-known technique](https://en.wikipedia.org/wiki/Approximations_of_%CF%80#Summing_a_circle's_area) for numerically estimating $\pi$ is to select a large number of random points within the unit square and count the fraction that fall within the unit circle. Since the area of the square is 1 and the area of the circle is $\pi / 4$, the fraction of points that fall in the circle (multiplied by 4) is a good approximation of $\pi$.

<center><img src="https://upload.wikimedia.org/wikipedia/commons/8/84/Pi_30K.gif" width="600">

© [User:nicoguaro](https://commons.wikimedia.org/wiki/User:Nicoguaro) / [Wikimedia Commons](https://commons.wikimedia.org/wiki/Main_Page) / [CC-BY-3.0](https://creativecommons.org/licenses/by/3.0/deed.en)
</center>

A nice property of this problem from the perspective of parallel programming is that each random point can be evaluated independently. We only need to know its coordinate to evaluate whether it falls within the circle; that is, given a point with coordinates $(x, y)$, all we need to do is check whether $x^2 + y^2 <= 1$. If it is, we increment our counter that measures the number of points in the circle. This can be done independently of every other point (although we do have to avoid race conditions in updating the counter).

With that in mind, let's see how this looks in CUDA (starting off with just a single GPU). We've provided a sample implementation; click on [code/monte_carlo_pi.cpp](code/monte_carlo_pi.cpp) to open it in a new tab and review the code. Note that this code is just meant for instructional purposes, it is not meant to be especially high performance. In particular:

- We're using the [device-side API](https://docs.nvidia.com/cuda/curand/device-api-overview.html) of [cuRAND](https://developer.nvidia.com/curand) to generate random numbers directly in the kernel. It's OK if you're unfamiliar with cuRAND, just know that every CUDA thread will have its own unique random numbers.
- We're having every thread only evaluate a single value, so the arithmetic intensity is quite low.
- We'll have a lot of atomic collisions while updating the `hits` counter.

Nevertheless, we can quickly estimate $\pi$ using one million sample points and we should get an error compared to the correct value of only about 0.05%.

To run the code, execute the next cell.

In [1]:
!nvcc -x cu -arch=sm_70 -rdc=true -o monte_carlo code/monte_carlo_pi.cpp
!./monte_carlo

Estimated value of pi = 3.14319
Error = 0.000507708


### Extending to multiple GPUs

A simple way to extend this example to multiple GPUs is to use a single host process that manages multiple GPUs. If we have *M* GPUs and *N* sample points to evaluate, we can distribute *N/M* to each GPU, and in principle calculate the result up to *M* times more quickly.

Look at [exercises/monte_carlo_mgpu_cuda.cpp](exercises/monte_carlo_mgpu_cuda.cpp) for an example of this. We've given you a few simple tasks to do in the code -- look for locations denoted by `FIXME`. Note that in this example we're giving each GPU a different seed for the random number generator so that they're each doing different work. As a result our answer will change a little.

If you get stuck, you can consult [the solution](solutions/monte_carlo_mgpu_cuda.cpp).

In [2]:
!nvcc -x cu -arch=sm_70 -rdc=true -o monte_carlo_mgpu_cuda solutions/monte_carlo_mgpu_cuda.cpp

In [3]:
%%time
!./monte_carlo_mgpu_cuda

Using 4 GPUs
Estimated value of pi = 3.14072
Error = 0.000277734
CPU times: user 18.4 ms, sys: 11.1 ms, total: 29.5 ms
Wall time: 2.04 s


### Peer access

CUDA uses a [universal virtual address (UVA) space](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#unified-virtual-address-space). All CUDA allocations (including both `cudaMalloc` and `cudaMallocHost`) that occur on CPUs and GPUs in this UVA space are guaranteed to have unique virtual addresses. This is, for example, what allows you to allocate pinned host memory with `cudaMallocHost` or `cudaHostAlloc` and take its address directly in device code (along with the virtual-to-physical address translation being fixed so that the GPU does not need to talk to the CPU's memory management unit). That is, in the UVA paradigm, CUDA knows which device a given address belongs to because by construction the same address is not used for different allocations on different devices.

<center><img src="images/UVA.png" width="1000"></center>

(Note the image above depicts the GPUs as connected via PCIe, but when UVA is supported it works over NVLink and/or NVSwitch as well.)

This also supports [direct access of peer GPU memory](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#peer-to-peer-memory-access), which is sometimes called "GPUDirect Peer-to-Peer (P2P)." From one GPU we can directly read from and write to an address on another GPU on the same server. (This is not true for every server; it depends on the system PCIe, NVLink, or NVSwitch topology. You can use the CUDA API call [cudaDeviceCanAcessPeer()](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__PEER.html) to check if this is possible on your configuration.) The only requirement is that you enable this peer access at the beginning of your program with [cudaDeviceEnablePeerAccess()](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__PEER.html).

<center><img src="images/GPUDirectP2P.png" width="1000"></center>

As a simple example of how this works, let's try this out on our application, in [exercises/monte_carlo_mgpu_cuda_peer.cpp](exercises/monte_carlo_mgpu_cuda_peer.cpp). Our strategy will be for every thread to update the *same* hits counter, rather than having one counter per GPU. We'll arbitrarily place this counter on GPU 0. This allows the application to look more like the original case (at the expense of increasing the number of possible atomic collisions on the counter). As before, look for `FIXME` in the code for the parts you should write yourself. You can consult [the solution](solutions/monte_carlo_mgpu_cuda_peer.cpp) if you need help. Check to make sure that we get the same result as above -- we're not doing different work, we're just updating the results to a different memory location, so the answer should be identical.

In [4]:
!nvcc -x cu -arch=sm_70 -rdc=true -o monte_carlo_mgpu_cuda_peer solutions/monte_carlo_mgpu_cuda_peer.cpp

In [5]:
%%time
!./monte_carlo_mgpu_cuda_peer

Estimated value of pi = 3.14072
Error = 0.000277734
CPU times: user 12.8 ms, sys: 16.4 ms, total: 29.2 ms
Wall time: 2.05 s


### Adding MPI

Managing all devices in a single program can be cumbersome. It often looks like what we've been doing so far, where we loop over all available devices and then take the same action (e.g. launch a kernel). The program can be simplified greatly with MPI (the [Message Passing Interface](https://en.wikipedia.org/wiki/Message_Passing_Interface)). When using MPI, we launch the *same* program multiple times independently (the [single-program, multiple-data](https://en.wikipedia.org/wiki/SPMD) paradigm). In the most common use case, we launch as many independent copies of the process as there are GPUs in your server, and each copy works with exactly one GPU.

<center><img src="images/MPI.png" width="1000"></center>

Each independent process has a unique numerical identifier associated with it (called its **rank**) as well as information about how many total processes are running. We can programmatically obtain the rank for each process (using `MPI_Comm_rank()`) and the number of processes (using `MPI_Comm_size()`). With this information, we can have each rank make independent processing decisions (while still using only one copy of the source code to achieve that). For example, we can (arbitrarily) set the GPU to be used with `cudaSetDevice()` to be equal to the rank (assuming there are at most the same number of ranks as GPUs).

With MPI, it's straightforward for every rank to independently do its *N / M* calculations and then sum them up, and indeed this is the most common way to write this style of program when using MPI. In [exercises/monte_carlo_mgpu_cuda_mpi.cpp](exercises/monte_carlo_mgpu_cuda_mpi.cpp) we've sketched out how this application would look with MPI (again, leaving a couple simple `FIXME` tasks for you to do).

Some differences include:

- MPI must be initialized and finalized as the first and last thing (respectively) in an MPI program.

```
    // Initialize MPI
    MPI_Init(&argc, &argv);
    ...
    // Finalize MPI
    MPI_Finalize();
```

- We select one GPU for each rank.

```
    // Each rank (arbitrarily) chooses the GPU corresponding to its rank
    int dev = rank;
    cudaSetDevice(dev);
```

- We sum up the results across all ranks and store the result in a single rank, which will arbitrarily do the final calculation and print. By convention, this is rank 0 (often called the "root" processor).

```
    // Accumulate the results across all ranks to the result on rank 0
    int* total_hits;
    total_hits = (int*) malloc(sizeof(int));

    int root = 0;
    MPI_Reduce(hits, total_hits, 1, MPI_INT, MPI_SUM, root, MPI_COMM_WORLD);
    
    if (rank == root) {
        // Calculate final value of pi and print out result
        ...
    }
```

Once you're ready to go, execute the next cell (note that we defined `NUM_DEVICES` earlier as the number of GPUs in this system). The parameter `-np` determines how many independent copies of the process (MPI ranks) are used.

In [8]:
!nvcc -ccbin=mpicxx -x cu -arch=sm_70 -rdc=true -o monte_carlo_mgpu_cuda_mpi solutions/monte_carlo_mgpu_cuda_mpi.cpp
!mpirun -np $NUM_DEVICES ./monte_carlo_mgpu_cuda_mpi

Estimated value of pi = 3.14072
Error = 0.000277734


### CUDA-aware MPI

MPI helped clean up much of the boilerplate we used when managing multiple devices explicitly. But we also gave up the benefit of multiple GPUs talking to each other directly. MPI is a [distributed memory parallel programming model](https://en.wikipedia.org/wiki/Distributed_memory), where each processor has its own (virtual) memory and address space, even if all ranks are on the same server and thus share the same physical memory. (This is typically contrasted with [shared memory parallel programming models](https://en.wikipedia.org/wiki/Shared_memory), where each processing thread has access to the same memory space, like [OpenMP](https://en.wikipedia.org/wiki/OpenMP), and also like traditional single-GPU CUDA programming where all threads have access to global memory.) So we copied the result for each GPU to the CPU and then summed the results on the CPU.

But as long as we're staying on a single server the rules of the [CUDA universal address space](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#unified-virtual-address-space) still hold, so all *CUDA* allocations result in virtual addresses that can be meaningfully shared across processes (even if the normal CPU dynamic memory allocations cannot be). As a result, it's possible for MPI to directly implement peer memory copies under the hood. For communication among remote servers this is not possible, but there are other technologies that allow direct GPU to GPU communication through a network interface, in particular [GPUDirect RDMA](https://docs.nvidia.com/cuda/gpudirect-rdma/index.html). Recognizing the value in leveraging these technologies for efficient communication, many MPI implementations (including OpenMPI) provide [CUDA-aware MPI](https://developer.nvidia.com/blog/introduction-cuda-aware-mpi/), which allows the programmer to provide an address to an MPI communication routine which may reside on a device. The MPI implementation is then free to use whatever scheme it desires to transmit the data from one GPU to another, including the use of GPUDirect P2P and GPUDirect RDMA where appropriate. (Note that [GPUDirect](https://developer.nvidia.com/gpudirect) refers to a family of technologies while CUDA-aware MPI refers to an API which may use those technologies under the hood, although it is common to see the two terms incorrectly conflated.)

<center><img src="images/GPUDirectRDMA.png" width="1000"></center>

So CUDA-aware MPI provides the benefit of simplified programming while retaining the performance benefit of avoiding unnecessary copies to CPU memory. With that in mind, one way to write the final reduction is:

```
    MPI_Reduce(d_hits, total_hits, 1, MPI_INT, MPI_SUM, root, MPI_COMM_WORLD);
```

where MPI automatically detects that the send buffer `d_hits` resides on the device while the receive buffer `total_hits` resides on the host and does the right thing behind the scenes to enable this copy. First, let's verify this works:

In [9]:
!nvcc -ccbin=mpicxx -x cu -arch=sm_70 -rdc=true -o monte_carlo_mgpu_cuda_mpi_cuda_aware exercises/monte_carlo_mgpu_cuda_mpi_cuda_aware.cpp
!mpirun -np $NUM_DEVICES ./monte_carlo_mgpu_cuda_mpi_cuda_aware

Estimated value of pi = 3.14072
Error = 0.000277734


Now, as a simple exercise, rewrite this application to do the reduction entirely in GPU memory, and then explicitly copy the result back to the host on rank 0 at the end. You can see the solution in [solutions/monte_carlo_mgpu_cuda_mpi_cuda_aware.cpp](solutions/monte_carlo_mgpu_cuda_mpi_cuda_aware.cpp).

In [10]:
!nvcc -ccbin=mpicxx -x cu -arch=sm_70 -rdc=true -o monte_carlo_mgpu_cuda_mpi_cuda_aware solutions/monte_carlo_mgpu_cuda_mpi_cuda_aware.cpp
!mpirun -np $NUM_DEVICES ./monte_carlo_mgpu_cuda_mpi_cuda_aware

Estimated value of pi = 3.14072
Error = 0.000277734


## Summary

You've now seen multiple methods for managing multiple GPUs to distribute a parallel workload, including looping over devices explicitly and using MPI to use multiple devices implicitly, and multiple methods for handling data transfer between GPUs, including direct peer memory access, peer to peer CUDA memory copies, and copies that go through the CPU. Hopefully this gives you a better feel for the pros and cons of each method. In particular, the CUDA-aware MPI method is very nice and can be highly performant, but we also have to return to the CPU to initiate the transfer. For many applications, in particular ones that hit strong scaling limits due to how efficient GPUs are at traditional calculations, the latency of returning to the CPU can be a significant performance penalty. Wouldn't it be nice if we could have the SPMD-like characteristics of MPI that result in highly understandable programmings, while retaining the potential performance benefit of initiating transfers directly from the kernel? Next, we're going to introduce NVSHMEM and demonstrate that it provides exactly that benefit.

<center><img src="images/NVSHMEM_versus_MPI.png" width="1000"></center>